In [ ]:
from typing import List, Optional
from langchain_huggingface import HuggingFaceEmbeddings
from deepeval.models import DeepEvalBaseEmbeddingModel

from greencompute_backend.services.llm.svc import EMBEDDINGS_MODEL

class CustomEmbeddingModel(DeepEvalBaseEmbeddingModel):
    def __init__(self):
        pass

    def load_model(self):
        return HuggingFaceEmbeddings(
			model_name=EMBEDDINGS_MODEL
		)

    def embed_text(self, text: str) -> List[float]:
        embedding_model = self.load_model()
        return embedding_model.embed_query(text)

    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        embedding_model = self.load_model()
        return embedding_model.embed_documents(texts)

    async def a_embed_text(self, text: str) -> List[float]:
        embedding_model = self.load_model()
        return await embedding_model.aembed_query(text)

    async def a_embed_texts(self, texts: List[str]) -> List[List[float]]:
        embedding_model = self.load_model()
        return await embedding_model.aembed_documents(texts)

    def get_model_name(self):
        "Custom HugginFace Embeddings Model"

In [1]:
from deepeval.synthesizer import Synthesizer
from dotenv import load_dotenv; load_dotenv()

synthesizer = Synthesizer(
	model="gpt-4o-mini",
	embedder=CustomEmbeddingModel(),
)
synthesizer.generate_goldens_from_docs(
    document_paths=['output.txt'],
)

Event loop is already running. Applying nest_asyncio patch to allow async execution...


✨ 🧩 ✨ Generating Contexts: 100%|██████████| 9/9 [00:04<00:00,  1.86it/s]


Utilizing 3 out of 52 chunks.

✨ Generating up to 6 goldens using DeepEval (using gpt-4o and text-embedding-3-small, use case=QA, method=docs):  50%|█████     | 3/6 [00:14<00:10,  3.60s/it]ERROR:root:OpenAI rate limit exceeded. Retrying: 1 time(s)...
ERROR:root:OpenAI rate limit exceeded. Retrying: 1 time(s)...
✨ Generating up to 6 goldens using DeepEval (using gpt-4o and text-embedding-3-small, use case=QA, method=docs): 100%|██████████| 6/6 [00:35<00:00,  5.89s/it]


✓ Generation finished 🎉! You can also run 'deepeval login' to generate and save goldens directly on Confident AI.

[Golden(input='Investigate how elevated RH levels mitigate ESD threats in data center environments.', actual_output=None, expected_output='Elevated relative humidity (RH) levels, such as 40% RH at the IT equipment intake, help mitigate electrostatic discharge (ESD) threats in data center environments. Maintaining higher humidity reduces the likelihood of ESD, which can damage sensitive electronic components. This is because dry air increases the potential for static electricity buildup. However, maintaining high RH levels can be energy-intensive, especially if electric humidifiers are used. Alternatives to manage ESD include using conductive flooring materials, implementing good cable grounding methods, and providing grounded wrist straps for technicians.', context=[' centers is often set relatively high (40% RH at the IT equipment intake is common) to guard against damage to the equipment due to electrostatic discharge (ESD). Maintaining this level of humidity is energy intensive if t

In [2]:
synthesizer.to_pandas().to_csv('output.csv', index=False)